In [ ]:
#Restaurant
import pandas as pd
from geopy.distance import geodesic

# Load the dataset containing restaurant locations
restaurant_data = pd.read_csv('zomatotest.csv', encoding='latin-1')

# Drop rows with missing or invalid latitude and longitude values
restaurant_data = restaurant_data.dropna(subset=['latitude', 'longitude'])
restaurant_data = restaurant_data[(restaurant_data['latitude'] >= -90) & (restaurant_data['latitude'] <= 90)]
restaurant_data = restaurant_data[(restaurant_data['longitude'] >= -180) & (restaurant_data['longitude'] <= 180)]

# Ask for the user's location
user_latitude = float(input("Enter your latitude: "))
user_longitude = float(input("Enter your longitude: "))
user_location = (user_latitude, user_longitude)

# Ask for user preferences
preferred_cuisine = input("Enter your preferred cuisine: ")
max_cost = float(input("Enter your maximum budget (cost): "))
min_rating = float(input("Enter your minimum rating: "))

# Function to calculate distance between two coordinates
def calculate_distance(restaurant_location):
    return geodesic(user_location, restaurant_location).kilometers

# Calculate distance for each restaurant
restaurant_data['Distance'] = restaurant_data.apply(lambda row: calculate_distance((row['latitude'], row['longitude'])), axis=1)

# Filter restaurants within 10 km radius
restaurants_within_radius = restaurant_data[restaurant_data['Distance'] <= 50]
restaurants_within_radius1 = restaurant_data[restaurant_data['Distance'] <= 10]

# Filter restaurants based on user preferences
filtered_restaurants = restaurants_within_radius1[
    (restaurants_within_radius1['cuisines'].str.contains(preferred_cuisine, case=False)) &
    (restaurants_within_radius1['average_cost_for_two'] <= max_cost) &
    (restaurants_within_radius1['aggregate_rating'] >= min_rating)
]

# Sort filtered restaurants based on rating and cost
sorted_restaurants = filtered_restaurants.sort_values(by=['aggregate_rating', 'average_cost_for_two'], ascending=[False, True])

if sorted_restaurants.empty:
    print("No restaurants match your criteria. Here are the top 10 nearest restaurants:")
    
    # Sort restaurants by distance
    nearest_restaurants = restaurants_within_radius.sort_values(by='Distance').head(10)
    
    for index, restaurant in nearest_restaurants.iterrows():
        print(f"{restaurant['name']} - Cuisine: {restaurant['cuisines']}, Cost: {restaurant['average_cost_for_two']}, Rating: {restaurant['aggregate_rating']}, Distance: {restaurant['Distance']} km")
else:
    # Get the top suggestion
    top_suggestion = sorted_restaurants.iloc[0]  # Assuming the first restaurant is the best suggestion
    
    # Print the details of the top suggestion
    print("Top Suggestion:")
    print("Name:", top_suggestion['name'])
    print("Cuisine:", top_suggestion['cuisines'])
    print("Cost:", top_suggestion['average_cost_for_two'])
    print("Rating:", top_suggestion['aggregate_rating'])
    print("Distance:", top_suggestion['Distance'], "km")

In [ ]:
import pandas as pd

# Load your original dataset
df_original = pd.read_csv('testcatf.csv', encoding='latin-1')
print("Original dataset columns:", df_original.columns)

# Ensure your original dataset has the necessary columns
required_columns = ['Predicted_Types', 'latitude', 'longitude', 'Categories', 'votes', 'Place','Place_Name']
for col in required_columns:
    if col not in df_original.columns:
        raise ValueError(f"Missing required column: {col}")

# Load the existing dataset
df_existing = pd.read_csv('Places.csv', encoding='latin-1')
print("Existing dataset columns:", df_existing.columns)

# Ensure the existing dataset has the required column 'Place'
if 'Place' not in df_existing.columns:
    raise ValueError("Missing required column: 'Place' in the existing dataset")

# Print first few rows of both datasets
print("Original dataset head:")
print(df_original.head())
print("Existing dataset head:")
print(df_existing.head())

# Select the specified columns from the original dataset
df_selected = df_original[required_columns]

# Merge the selected columns with the existing dataset
df_merged = pd.merge(df_existing, df_selected, on='Place', how='left')

# Print the head of the merged dataset
print("Merged dataset head:")
print(df_merged.head())

# Save the resulting merged dataset to a new CSV file
df_merged.to_csv('merged_dataset.csv', index=False)

print("Merged dataset saved to 'merged_dataset.csv'")
